In [254]:
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='serif', size=80)
import pylab
%matplotlib inline
pylab.rcParams['figure.figsize'] = 8, 8

In [255]:
import ROOT
from root_numpy import root2array, tree2array, fill_hist
from sklearn import linear_model, ensemble
from ROOT import TFile, TTree, TBranch
from array import array

In [256]:
#rfile = ROOT.TFile('/Disk/ds-sopa-group/PPE/titus/ts-WChRecoSandBox/scripts/editing_ene/outputs/nu_numu_1000_1039_CCQE_12in_energy_studies_recoquant_tree.root')
rfile = ROOT.TFile('/Disk/ds-sopa-group/PPE/titus/ts-WChRecoSandBox/scripts/editing_ene/outputs/nu_numu_1000_1039_CCQE_12in_energy_studies_recoquant_tree_NEWlookupsB_for_training.root')
intree = rfile.Get('nu_eneNEW')

In [257]:
f = ROOT.TFile( 'enerReco.root', 'recreate' )
tr = ROOT.TTree( "tr", "tree with histos" )
arr=tree2array(intree)

TypeError: tree must be a ROOT.TTree

In [ ]:
arr2=arr[['total_hits2','total_ring_PEs2','recoDWallR2','recoDWallZ2','lambda_max_2']]#,'hits_pot_length2']]
arr2_n=arr2.view(arr2.dtype[0]).reshape(arr2.shape + (-1,))
arr3=arr['trueKE']

In [ ]:
clf = linear_model.SGDRegressor()
clf.fit(arr2_n,arr3)
clf

In [ ]:
plt.scatter(arr3,clf.predict(arr2_n)-arr3)
plt.xlabel("trueKE [MeV]")
plt.ylabel("recoKE - trueKE [MeV]")


In [ ]:
a = array( 'd', [ 0 ] )
b = array( 'f', [ 0 ] )
c = array( 'f', [ 0 ] )
#
#tr.Branch('truKE', a, 'trueKE/D')
#tr.Branch('recoKEL',b,'recoKEL/F')
#tr.Branch('recoKEGB',c, 'recoKEGB/F')
maxn = 10
n = array( 'i', [ 0 ] )
d = array( 'f', maxn*[ 0. ] )
tr.Branch( 'n', n, 'n/I' )
#tr.Branch( 'myval', d, 'myval[mynum]/F' )

In [ ]:
chain = ROOT.TChain('nu_eneNEW')
for i in range(1040,1099):
    if i==1051: i=i+1
    if i==1075: i=i+1    
    chain.Add('/Disk/ds-sopa-group/PPE/titus/ts-WChRecoSandBox/scripts/editing_ene/outputs/nu_numu_'+str(i)+'_CCQE_12in_energy_studies_recoquant_tree_NEWlookupsB_for_training.root')
              #CCQE_12in_energy_studies_recoquant_tree_NEWlookups.root')
test_data = tree2array(chain)

In [ ]:
test_data_reduced = test_data[['total_hits2','total_ring_PEs2','recoDWallR2','recoDWallZ2','lambda_max_2']]#,'hits_pot_length2']]
test_data_reduced_n = test_data_reduced.view(test_data_reduced.dtype[0]).reshape(test_data_reduced.shape + (-1,))
test_data_trueKE = test_data['trueKE']
#tr.Branch("trueKE", test_data_trueKE, "trueKE/D")
#tr.Fill()

In [ ]:
plt.scatter(test_data_trueKE,clf.predict(test_data_reduced_n)-test_data_trueKE)
plt.ylim((-5000,3000))
plt.xlabel("trueKE [MeV]")
plt.ylabel("recoKE - trueKE [MeV]")

In [ ]:
plt.scatter(test_data_trueKE,(clf.predict(test_data_reduced_n)-test_data_trueKE)/test_data_trueKE)
plt.ylim((0,1))
plt.xlabel("trueKE [MeV]")
plt.ylabel("DeltaE/E")
res_twod_SGD = np.dstack((test_data_trueKE, (clf.predict(test_data_reduced_n)-test_data_trueKE)/test_data_trueKE))

In [ ]:
params = {'n_estimators': 1000, 'max_depth': 10, 'min_samples_split': 1,
          'learning_rate': 0.01, 'loss': 'lad'}
net = ensemble.GradientBoostingRegressor(**params)
net.fit(arr2_n,arr3)
net

In [ ]:
plt.scatter(arr3,net.predict(arr2_n)-arr3,c='r')
plt.xlabel("trueKE [MeV]")
plt.ylabel("recoKE - trueKE [MeV]")
plt.ylim(-50,50)

In [ ]:
plt.scatter(test_data_trueKE,(net.predict(test_data_reduced_n)-test_data_trueKE), c='r')
plt.xlabel("trueKE [MeV]")
plt.ylabel("recoKE - trueKE [MeV]")
matrix = np.dstack((test_data_trueKE, (net.predict(test_data_reduced_n)-test_data_trueKE)))

In [ ]:
plt.scatter(test_data_trueKE,((net.predict(test_data_reduced_n)-test_data_trueKE)/test_data_trueKE),c='r')
plt.xlabel("trueKE [MeV]")
plt.ylabel("DeltaE/E")
plt.ylim(-2,2)
twod_GBR_abs = np.dstack((test_data_trueKE, np.abs(net.predict(test_data_reduced_n)-test_data_trueKE)/test_data_trueKE))

In [ ]:
a=test_data_trueKE
b=clf.predict(test_data_reduced_n)
c=net.predict(test_data_reduced_n)
for i in range(25):
   n[0] = min(i,maxn)
   for j in range(n[0]):
      d[j] = i*0.1+j
   tr.Fill()
#tr.Fill()

In [ ]:
tr.Write()
#hist_trueKE_zoom.Write()
f.Write()
f.Close()